# Agentic AI Kaggle Competition Assistant (Enhanced)

This notebook runs an AI assistant for Kaggle competitions, demonstrating key concepts like **Custom Tools**, **Session Management**, and **Observability**.

**Instructions:**
1. Add the "Meta Kaggle" and "Meta Kaggle Code" datasets.
2. Add your `GOOGLE_API_KEY` as a secret.
3. Run the cells below.

## 1. Setup and Imports

In [1]:
!pip install -q google-generativeai pandas tqdm python-dotenv
import os, re, json, pandas as pd
from tqdm import tqdm
from datetime import datetime
from typing import Dict, List, Any
from dataclasses import dataclass, field
import google.generativeai as genai
from google.generativeai.types import FunctionDeclaration, Tool

GOOGLE_API_KEY = None
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    print("API Key loaded from Kaggle Secrets.")
except (ImportError, Exception):
    print("Could not load from Kaggle Secrets, checking environment variables.")
    from dotenv import load_dotenv
    load_dotenv()
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google AI configured successfully.")
else:
    print("Error: GOOGLE_API_KEY not found.")

API Key loaded from Kaggle Secrets.
Google AI configured successfully.


## 2. Agent Source Code Modules

In [14]:
!git clone https://github.com/lmassaron/Agentic-AI-Kaggle-Competition-Assistant.git

Cloning into 'Agentic-AI-Kaggle-Competition-Assistant'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 2), reused 23 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 245.07 KiB | 20.42 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [ ]:
import sys
import os
import glob
import importlib

project_root = os.path.abspath('Agentic-AI-Kaggle-Competition-Assistant')

if project_root not in sys.path:
    sys.path.append(project_root)

src_folder = os.path.join(project_root, 'src')
py_files = glob.glob(os.path.join(src_folder, '*.py'))

for file_path in py_files:
    filename = os.path.basename(file_path)[:-3]
    
    if filename == "__init__":
        continue
        
    module_name = f"src.{filename}"
    
    print(f"Importing {module_name}...")
    try:
        module = importlib.import_module(module_name)
        # Optional: Add to global namespace if you really need to use functions directly
        # globals().update(vars(module)) 
    except Exception as e:
        print(f"Failed to import {module_name}: {e}")

## 3. Initialize and Run the Agent

In [ ]:
if GOOGLE_API_KEY and datasets:
    agent = KaggleAgent(api_key=GOOGLE_API_KEY)
    print("Kaggle Assistant is ready!")
else:
    print("Agent not initialized. Check API key and dataset paths.")

In [12]:
response = agent.run("Analyze the competition Hull Tactical - Market Prediction at https://www.kaggle.com/competitions/hull-tactical-market-prediction")
print(f"Assistant Response:{response}")

Analyzing competition at URL: https://www.kaggle.com/competitions/hull-tactical-market-prediction
Assistant Response:Successfully fetched content from https://www.kaggle.com/competitions/hull-tactical-market-prediction. Analysis would go here.


In [8]:
response = agent.run("Can you find similar competitions such as Hull Tactical - Market Prediction")
print(f"Assistant Response:{response}")

Finding similar competitions for query: 'Hull Tactical - Market Prediction', metric: 'None'
Assistant Response:Found similar competitions: [{'Id': 111543, 'Title': 'Hull Tactical - Market Prediction', 'EvaluationAlgorithmName': 'Hull Competition Sharpe', 'TotalSubmissions': 29212}]


In [11]:
response = agent.run("Can you analyze a competition page Hull Tactical - Market Prediction")
print(f"Assistant Response:{response}")

Assistant Response:I can only analyze a competition page if you provide a URL. Could you please provide the URL for "Hull Tactical - Market Prediction"?


## 4. Demonstrating Key Concepts

In [9]:
if 'agent' in locals() and agent:
    print("--- Session & Memory: Conversation History ---")
    print(json.dumps(agent.memory.get_full_history(), indent=2))
    print("--- Observability: Session Statistics ---")
    print(json.dumps(agent.get_session_stats(), indent=2))

--- Session & Memory: Conversation History ---
[
  {
    "role": "user",
    "content": "Analyze the competition Hull Tactical - Market Prediction at https://www.kaggle.com/competitions/hull-tactical-market-prediction",
    "timestamp": "2025-11-29T22:57:26.638848"
  },
  {
    "role": "assistant",
    "content": "Successfully fetched content from https://www.kaggle.com/competitions/hull-tactical-market-prediction. Analysis would go here.",
    "timestamp": "2025-11-29T22:57:29.093328"
  },
  {
    "role": "user",
    "content": "Can you find similar competitions such as Hull Tactical - Market Prediction",
    "timestamp": "2025-11-29T22:57:29.104565"
  },
  {
    "role": "assistant",
    "content": "Found similar competitions: [{'Id': 111543, 'Title': 'Hull Tactical - Market Prediction', 'EvaluationAlgorithmName': 'Hull Competition Sharpe', 'TotalSubmissions': 29212}]",
    "timestamp": "2025-11-29T22:57:30.521353"
  }
]
--- Observability: Session Statistics ---
{
  "agent_stats": {
 